In [1]:
!nvidia-smi

Tue Apr 29 23:22:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
# Connet to google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
!cd '/content/drive/MyDrive/CLIP_video_training'

In [5]:
!git clone https://github.com/sijiasiga/CLIP_video.git '/content/drive/MyDrive/CLIP_video_training/CLIP_video_5'
# !git clone https://github.com/sijiasiga/CLIP_video.git

Cloning into '/content/drive/MyDrive/CLIP_video_training/CLIP_video_5'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 230 (delta 29), reused 0 (delta 0), pack-reused 187 (from 1)
Receiving objects: 100% (230/230), 1.51 MiB | 22.70 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [6]:
# From CLIP
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install opencv-python boto3 requests pandas

/bin/bash: line 1: conda: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 136.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 11.4 MB/s eta 0:00:00


In [ ]:
# # Comment if the file exits
# !wget -O /content/drive/MyDrive/Data/msrvtt_data.zip https://github.com/ArrowLuo/CLIP4Clip/releases/download/v0.0/msrvtt_data.zip
# !unzip /content/drive/MyDrive/Data/msrvtt_data.zip -d /content/drive/MyDrive/Data/

In [7]:
DATA_PATH = "/content/drive/MyDrive/CLIP_video_training/Data/msrvtt_data"  # Directory of MSRVTT data
VIDEO_PATH = "/content/drive/MyDrive/CLIP_video_training/Data/video"    # Directory of MSRVTT raw video

In [8]:
import pandas as pd

# Load training dataset
train_path = f"{DATA_PATH}/MSRVTT_train.9k.csv"
df = pd.read_csv(train_path)

print("Size: ", len(df))
print("Column names", df.columns.tolist())
display(df.head())

# Choose a subset
def sample_subset(df, frac=0.1, random_state=42):
    return df.sample(frac=frac, random_state=random_state).reset_index(drop=True)

# Set fraction to 0.01
subset_df = sample_subset(df, frac=0.1)
print("Size of subset：", len(subset_df))
display(subset_df.head())

# Save the subset
subset_path = f"{DATA_PATH}/MSRVTT_train.subset.csv"
subset_df.to_csv(subset_path, index=False)

print("Subset CSV saved to:", subset_path)


Size:  9000
Column names ['video_id']


,video_id
0,video0
1,video1
2,video2
3,video3
4,video4


Size of subset： 900


,video_id
0,video8405
1,video1162
2,video582
3,video4081
4,video9139


Subset CSV saved to: /content/drive/MyDrive/CLIP_video_training/Data/msrvtt_data/MSRVTT_train.subset.csv


In [9]:
import torch
print("Current device:", torch.cuda.get_device_name() if torch.cuda.is_available() else "CPU only")

Current device: NVIDIA A100-SXM4-40GB


In [11]:
!wget -P /content/drive/MyDrive/CLIP_video_training/CLIP_video_5/modules https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt

--2025-04-29 23:23:58--  https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.253.59, 2620:1ec:bdf::59
Connecting to openaipublic.azureedge.net (openaipublic.azureedge.net)|13.107.253.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353976522 (338M) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/CLIP_video_training/CLIP_video_5/modules/ViT-B-32.pt’

ViT-B-32.pt         100%[===================>] 337.58M   105MB/s    in 3.2s    

2025-04-29 23:24:01 (105 MB/s) - ‘/content/drive/MyDrive/CLIP_video_training/CLIP_video_5/modules/ViT-B-32.pt’ saved [353976522/353976522]



In [12]:
torch.cuda.empty_cache()

In [ ]:
# # Updata changes to files. Comment if the files are not changed
# !cd CLIP_video && git pull

In [ ]:
# !pip install ffmpeg
# !python /content/CLIP_video/preprocess/compress_video.py --input_root "/content/drive/MyDrive/IDL/IDL Project/Data/video" --output_root "/content/drive/MyDrive/IDL/IDL Project/Data/compressed_video"

In [13]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

!python /content/drive/MyDrive/CLIP_video_training/CLIP_video_5/main_task_retrieval.py \
  --do_train \
  --num_thread_reader 8 \
  --epochs 20 \
  --batch_size 64 \
  --train_csv  "$DATA_PATH"/MSRVTT_train.subset.csv \
  --val_csv    "$DATA_PATH"/MSRVTT_JSFUSION_test.csv \
  --data_path  "$DATA_PATH"/MSRVTT_data.json \
  --features_path "$VIDEO_PATH" \
  --output_dir /content/drive/MyDrive/CLIP_video_training/CLIP_video_5/ckpts/ckpt_msrvtt_retrieval_looseType \
  --lr 1e-4 \
  --max_words 32 \
  --max_frames 8 \
  --batch_size_val 8 \
  --datatype msrvtt \
  --expand_msrvtt_sentences \
  --feature_framerate 1 \
  --coef_lr 1e-3 \
  --freeze_layer_num 6 \
  --slice_framepos 5 \
  --loose_type \
  --linear_patch 2d \
  --sim_header meanP \
  --pretrained_clip_name ViT-B/32 \
  --fp16 \
  --fp16_opt_level O1


04/29/2025 23:26:03 - INFO -   Effective parameters:
04/29/2025 23:26:03 - INFO -     <<< batch_size: 64
04/29/2025 23:26:03 - INFO -     <<< batch_size_val: 8
04/29/2025 23:26:03 - INFO -     <<< cache_dir: 
04/29/2025 23:26:03 - INFO -     <<< coef_lr: 0.001
04/29/2025 23:26:03 - INFO -     <<< cross_model: cross-base
04/29/2025 23:26:03 - INFO -     <<< cross_num_hidden_layers: 4
04/29/2025 23:26:03 - INFO -     <<< data_path: /content/drive/MyDrive/CLIP_video_training/Data/msrvtt_data/MSRVTT_data.json
04/29/2025 23:26:03 - INFO -     <<< datatype: msrvtt
04/29/2025 23:26:03 - INFO -     <<< do_eval: False
04/29/2025 23:26:03 - INFO -     <<< do_lower_case: False
04/29/2025 23:26:03 - INFO -     <<< do_pretrain: False
04/29/2025 23:26:03 - INFO -     <<< do_train: True
04/29/2025 23:26:03 - INFO -     <<< epochs: 20
04/29/2025 23:26:03 - INFO -     <<< eval_frame_order: 0
04/29/2025 23:26:03 - INFO -     <<< expand_msrvtt_sentences: True
04/29/2025 23:26:03 - INFO -     <<< feature_